<a href="https://colab.research.google.com/github/vedanshipathak/Mental-Health-Support-Chatbot/blob/main/MentalHealthSupport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.5 MB/s eta 0:00:00


In [ ]:
from datasets import get_dataset_config_names, load_dataset

# List all available subsets in the dataset
dataset_name = "Psychotherapy-LLM/CBT-Bench"
available_subsets = get_dataset_config_names(dataset_name)
print("Available subsets:", available_subsets)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.90k [00:00<?, ?B/s]

Available subsets: ['qa_test', 'qa_seed', 'distortions_test', 'distortions_seed', 'core_major_test', 'core_major_seed', 'core_fine_test', 'core_fine_seed', 'dp_ref_exe_1', 'dp_ref_exe_2', 'dp_ref_exe_3', 'dp_ref_exe_4', 'dp_ref_exe_5', 'dp_ref_exe_6', 'dp_ref_exe_7', 'dp_ref_exe_8', 'dp_ref_exe_9', 'dp_ref_exe_10', 'dp_llama8b_exe_1', 'dp_llama8b_exe_2', 'dp_llama8b_exe_3', 'dp_llama8b_exe_4', 'dp_llama8b_exe_5', 'dp_llama8b_exe_6', 'dp_llama8b_exe_7', 'dp_llama8b_exe_8', 'dp_llama8b_exe_9', 'dp_llama8b_exe_10', 'dp_llama405b_exe_1', 'dp_llama405b_exe_2', 'dp_llama405b_exe_3', 'dp_llama405b_exe_4', 'dp_llama405b_exe_5', 'dp_llama405b_exe_6', 'dp_llama405b_exe_7', 'dp_llama405b_exe_8', 'dp_llama405b_exe_9', 'dp_llama405b_exe_10', 'dp-pairwise-comparison']


In [ ]:
# Load a specific subset (e.g., 'dp_ref_exe_1')
subset_name = "dp_ref_exe_1"
dataset = load_dataset(dataset_name, subset_name)

# Check structure
print(dataset)
print(dataset['train'][0])  # Print first example


CBT-DP%2Fdp-reference-exe-1.json:   0%|          | 0.00/9.49k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'level', 'client_statement', 'response'],
        num_rows: 14
    })
})
{'id': '1_b_1', 'level': 'beginner', 'client_statement': '[Curious] Is this approach helpful for people like me?', 'response': 'Yes, this approach is generally quite helpful for people like you. This is a well-researched\napproach with substantial support for its effectiveness across a broad range of clients and\nproblems. Of course, each person is unique and experiences things differently. So, we’ll\ntailor what we do to best fit your needs and preferences, and make sure that we prioritize\nchecking in to see what seems to be working or not working for you.'}


In [ ]:
import pandas as pd
from datasets import Dataset, load_dataset
from sklearn.model_selection import train_test_split

# Dataset name
dataset_name = "Psychotherapy-LLM/CBT-Bench"

# All available subsets (from your provided list)
subset_names = [
    "qa_test", "qa_seed", "distortions_test", "distortions_seed",
    "core_major_test", "core_major_seed", "core_fine_test", "core_fine_seed",
    "dp_ref_exe_1", "dp_ref_exe_2", "dp_ref_exe_3", "dp_ref_exe_4", "dp_ref_exe_5",
    "dp_ref_exe_6", "dp_ref_exe_7", "dp_ref_exe_8", "dp_ref_exe_9", "dp_ref_exe_10",
    "dp_llama8b_exe_1", "dp_llama8b_exe_2", "dp_llama8b_exe_3", "dp_llama8b_exe_4",
    "dp_llama8b_exe_5", "dp_llama8b_exe_6", "dp_llama8b_exe_7", "dp_llama8b_exe_8",
    "dp_llama8b_exe_9", "dp_llama8b_exe_10", "dp_llama405b_exe_1", "dp_llama405b_exe_2",
    "dp_llama405b_exe_3", "dp_llama405b_exe_4", "dp_llama405b_exe_5", "dp_llama405b_exe_6",
    "dp_llama405b_exe_7", "dp_llama405b_exe_8", "dp_llama405b_exe_9", "dp_llama405b_exe_10",
    "dp-pairwise-comparison"
]

# Initialize an empty DataFrame
all_data = pd.DataFrame()

# Load and merge all subsets
for subset in subset_names:
    try:
        dataset = load_dataset(dataset_name, subset)
        df = pd.DataFrame(dataset['train'])  # Convert to DataFrame

        # Identify the correct columns
        if 'client_statement' in df.columns and 'response' in df.columns:
            df = df[['client_statement', 'response']]
        elif 'question' in df.columns and 'answer' in df.columns:
            df = df.rename(columns={'question': 'client_statement', 'answer': 'response'})
        elif 'input' in df.columns and 'output' in df.columns:
            df = df.rename(columns={'input': 'client_statement', 'output': 'response'})
        else:
            print(f"Skipping {subset}: No valid client-response structure found.")
            continue

        # Merge into one dataset
        all_data = pd.concat([all_data, df], ignore_index=True)

        print(f"Loaded subset: {subset}, Total records: {len(df)}")

    except Exception as e:
        print(f"Error loading {subset}: {e}")

# Shuffle dataset to ensure randomness
all_data = all_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Check final dataset size
print(f"Final combined dataset size: {len(all_data)}")

# Split into training (80%) and validation (20%) sets
train_df, val_df = train_test_split(all_data, test_size=0.2, random_state=42)

# Convert back to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

print(f"Final Training set size: {len(train_dataset)}, Validation set size: {len(val_dataset)}")



qa_test.json:   0%|          | 0.00/103k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/220 [00:00<?, ? examples/s]

Skipping qa_test: No valid client-response structure found.


qa_seed.json:   0%|          | 0.00/23.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/47 [00:00<?, ? examples/s]

...

Generating train split:   0%|          | 0/15 [00:00<?, ? examples/s]

Loaded subset: dp_llama405b_exe_10, Total records: 15


dp-pairwise-comparison.json:   0%|          | 0.00/1.15M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/468 [00:00<?, ? examples/s]

Skipping dp-pairwise-comparison: No valid client-response structure found.
Final combined dataset size: 468
Final Training set size: 374, Validation set size: 94


In [ ]:
print(train_dataset.to_pandas().head())  # Show first few rows


                                    client_statement  \
0  [Anxious] I know the job interview is going to...   
1  [Helpless] I was abused when I was a child, an...   
2  [Relieved] I’m glad you could see my reluctanc...   
3  [Irritated] Well, I guess the homework you’re ...   
4  [Frustrated] To be honest, the last couple of ...   

                                            response  __index_level_0__  \
0  How do you typically determine whether or not ...                304   
1  I imagine that feels pretty overwhelming. Some...                 22   
2  I'm glad we could have this conversation, and ...                421   
3  It sounds like you're feeling frustrated with ...                 46   
4  It's completely understandable that you're fee...                172   

                                           input_ids  \
0  [784, 188, 29, 226, 2936, 908, 27, 214, 8, 613...   
1  [784, 29582, 924, 908, 27, 47, 3, 28493, 116, ...   
2  [784, 1649, 1896, 162, 26, 908, 27, 22, 5

In [ ]:
from transformers import T5ForConditionalGeneration

# Now you can instantiate the model
model = T5ForConditionalGeneration.from_pretrained('t5-base')

tokenizer = T5Tokenizer.from_pretrained('t5-base')


# Tokenization function
def tokenize_data(example):
    input_encodings = tokenizer(
        example['client_statement'], padding="max_length", truncation=True, max_length=128
    )
    target_encodings = tokenizer(
        example['response'], padding="max_length", truncation=True, max_length=128
    )
    return {
        "input_ids": input_encodings["input_ids"],
        "attention_mask": input_encodings["attention_mask"],
        "labels": target_encodings["input_ids"],
    }

# Apply tokenization
train_dataset = train_dataset.map(tokenize_data)
val_dataset = val_dataset.map(tokenize_data)

print("✅ Tokenization complete.")


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/374 [00:00<?, ? examples/s]

Map:   0%|          | 0/94 [00:00<?, ? examples/s]

✅ Tokenization complete.


In [ ]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments, EncoderDecoderCache

# Load pre-trained T5 model
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=2,  # Keep only the latest 2 checkpoints
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Start training
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.715500,1.645443
2,1.501500,1.414421
3,1.343400,1.353732
4,1.268500,1.329308
5,1.220800,1.314815
6,1.189500,1.303762
7,1.165800,1.296995
8,1.142100,1.294463
9,1.129000,1.292442
10,1.124500,1.291629


TrainOutput(global_step=470, training_loss=1.4800539381960605, metrics={'train_runtime': 287.083, 'train_samples_per_second': 13.028, 'train_steps_per_second': 1.637, 'total_flos': 569375627673600.0, 'train_loss': 1.4800539381960605, 'epoch': 10.0})

In [ ]:
trainer.evaluate()


{'eval_loss': 1.2916289567947388,
 'eval_runtime': 1.8748,
 'eval_samples_per_second': 50.138,
 'eval_steps_per_second': 6.401,
 'epoch': 10.0}

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:
import csv
import json
from transformers import pipeline


def generate_response(client_input):
    model.eval()  # Set model to evaluation mode

    # Revised prompt: use dialogue format to encourage a distinct therapist response.
    formatted_input = f"Patient: {client_input}\nTherapist: "
    input_ids = tokenizer(formatted_input, return_tensors="pt").input_ids.to(device)

    output = model.generate(
        input_ids,
        max_length=150,
        num_beams=7,
        temperature=1.3,        # Increased for more creative outputs
        top_p=0.85,             # Adjusted for diversity
        top_k=50,
        repetition_penalty=4.5, # Stronger penalty to avoid repetition
        do_sample=True,
        early_stopping=True,
        num_return_sequences=1
    )

    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Optional post-processing to remove any echoing of the input.
    if response.startswith(client_input):
        response = response[len(client_input):].strip()
    return response if response else "🤖 Sorry, I didn't understand that."




# Define a list of test prompts to simulate real-world inputs.
test_prompts = [
    "I'm feeling very sad.",
    "What should I do if I'm stressed?",
    "I'm anxious about my future.",
    "I feel overwhelmed by everything.",
    "I'm having a tough day at work.",
    "I don't know how to cope with my anxiety.",
    "I feel alone and hopeless."
]

# Create an empty list to store evaluation results.
evaluation_results = []

# Generate responses for each prompt using your chatbot.
for prompt in test_prompts:
    response = generate_response(prompt)  # Your existing function
    evaluation_results.append({"prompt": prompt, "response": response})

# Save the results to a CSV file for review.
with open("qualitative_evaluation.csv", "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["prompt", "response"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for entry in evaluation_results:
        writer.writerow(entry)

# Optionally, perform sentiment analysis on the responses to get an extra qualitative metric.
sentiment_analyzer = pipeline("sentiment-analysis")
for entry in evaluation_results:
    sentiment = sentiment_analyzer(entry["response"])
    entry["sentiment"] = sentiment  # sentiment is a list of dicts

# Print out the results in JSON format for easy review.
print(json.dumps(evaluation_results, indent=2))


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


[
  {
    "prompt": "I'm feeling very sad.",
    "response": "Therapist: It sounds like you're feeling very sad right now. Can you tell me more about what's making you feel so sad?",
    "sentiment": [
      {
        "label": "NEGATIVE",
        "score": 0.9976403713226318
      }
    ]
  },
  {
    "prompt": "What should I do if I'm stressed?",
    "response": "Therapist: I'd like to hear from you if you feel stressed. Let's break down the stress into manageable steps.",
    "sentiment": [
      {
        "label": "POSITIVE",
        "score": 0.9949304461479187
      }
    ]
  },
  {
    "prompt": "I'm anxious about my future.",
    "response": "Therapist: I understand your anxiety about your future. Can you tell me more about what's making you feel anxious about your future?",
    "sentiment": [
      {
        "label": "NEGATIVE",
        "score": 0.5596469640731812
      }
    ]
  },
  {
    "prompt": "I feel overwhelmed by everything.",
    "response": "Therapist: I feel overwhel

In [ ]:
def generate_response(client_input):
    model.eval()  # Set model to evaluation mode

    # Using a dialogue-like prompt
    formatted_input = f"Patient: {client_input}\nTherapist:"
    input_ids = tokenizer(formatted_input, return_tensors="pt").input_ids.to(device)

    output = model.generate(
        input_ids,
        max_length=150,
        num_beams=7,
        temperature=1.3,        # Increased temperature for more diversity
        top_p=0.85,             # Adjusted top_p for diversity
        top_k=50,
        repetition_penalty=4.5,
        do_sample=True,
        early_stopping=True,
        num_return_sequences=1
    )

    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Optional post-processing to remove unwanted role labels
    def clean_response(resp):
        # Remove common role labels if present
        resp = resp.replace("Patient:", "").replace("Therapy:", "Therapist:")
        # Remove duplicate role markers if found
        if "Therapist:" in resp:
            parts = resp.split("Therapist:")
            # Keep only the part after the first occurrence of "Therapist:"
            resp = parts[1].strip() if len(parts) > 1 else resp
        return resp

    cleaned_response = clean_response(response)
    return cleaned_response if cleaned_response else "🤖 Sorry, I didn't understand that."

# Test the updated function with evaluation
print("🗣 AI Therapist:", generate_response("I'm feeling very sad."))
print("🗣 AI Therapist:", generate_response("What should I do if I'm stressed?"))
print("🗣 AI Therapist:", generate_response("I'm anxious about my future."))
print("🗣 AI Therapist:", generate_response("I feel overwhelmed by everything."))
print("🗣 AI Therapist:", generate_response("I'm having a tough day at work."))
print("🗣 AI Therapist:", generate_response("I don't know how to cope with my anxiety."))
print("🗣 AI Therapist:", generate_response("I feel alone and hopeless."))


🗣 AI Therapist: I understand your feelings. It sounds like you're feeling very sad. Let's break it down together and see if we can work together to help you feel better.
🗣 AI Therapist: What can I do if I feel stressed?  What should I do if I'm stressed?
🗣 AI Therapist: It sounds like you're feeling anxious about your future. Let's explore this further. Can you tell me more about what concerns you have about your future?
🗣 AI Therapist: Can you tell me more about what makes you feel overwhelmed?
🗣 AI Therapist: It sounds like you're having a hard day at work. What can I do to help you cope?
🗣 AI Therapist: It sounds like you're feeling overwhelmed with your anxiety. Can you tell me more about what might be causing it?
🗣 AI Therapist: It sounds like you're feeling overwhelmed and hopeless. Can you tell me more about what it feels like to feel alone and hopeless?


In [ ]:
print("🗣 AI Therapist:", generate_response("Hello, how are you?"))
print("🗣 AI Therapist:", generate_response("I'm feeling very sad."))
print("🗣 AI Therapist:", generate_response("What should I do if I'm stressed?"))

🗣 AI Therapist: Patient: Hello, how are you doing? Therapist: Can you tell me more about what's going on with your life right now?
🗣 AI Therapist: It sounds like you're feeling very sad. Therapy: Can you tell me more about what's making you feel so sad?
🗣 AI Therapist: Let's break it down together. What can I do if I feel stressed?


In [ ]:
from transformers import pipeline
import json

# Initialize the zero-shot classification pipeline.
context_classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli"
)

# Tweaked candidate labels for more nuanced evaluation.
candidate_labels = [
    "empathetic",
    "caring",
    "supportive",
    "insightful",
    "neutral",
    "uninformative",
    "unhelpful"
]

# Evaluate each response using the zero-shot classifier.
for entry in evaluation_results:
    classification = context_classifier(entry["response"], candidate_labels)
    entry["contextual_evaluation"] = classification

# Print out the updated evaluation results with context-aware analysis.
print(json.dumps(evaluation_results, indent=2))



Device set to use cuda:0


[
  {
    "prompt": "I'm feeling very sad.",
    "response": "Therapist: It sounds like you're feeling very sad right now. Can you tell me more about what's making you feel so sad?",
    "sentiment": [
      {
        "label": "NEGATIVE",
        "score": 0.9976403713226318
      }
    ],
    "contextual_evaluation": {
      "sequence": "Therapist: It sounds like you're feeling very sad right now. Can you tell me more about what's making you feel so sad?",
      "labels": [
        "empathetic",
        "caring",
        "insightful",
        "neutral",
        "supportive",
        "uninformative",
        "unhelpful"
      ],
      "scores": [
        0.5990208387374878,
        0.2413983941078186,
        0.07997582107782364,
        0.02810671180486679,
        0.027744334191083908,
        0.015593795105814934,
        0.008160054683685303
      ]
    }
  },
  {
    "prompt": "What should I do if I'm stressed?",
    "response": "Therapist: I'd like to hear from you if you feel str

In [ ]:
def generate_response(client_input):
    model.eval()  # Set model to evaluation mode

    # Use a prompt that instructs the model to provide a supportive, original response
    formatted_input = f"Therapist: {client_input}\nAdvice:"
    # formatted_input = f"Therapist: When someone says, '{client_input}', what supportive advice would you offer?"


    # Tokenize and move input to the device
    input_ids = tokenizer(formatted_input, return_tensors="pt").input_ids.to(device)

    # Generate response with tuned decoding parameters
    output = model.generate(
        input_ids,
        max_length=150,
        num_beams=7,
        temperature=1.2,
        top_p=0.9,
        top_k=50,
        repetition_penalty=4.0,
        do_sample=True,
        early_stopping=True,
        num_return_sequences=3  # Generate 3 responses
    )

    # Decode and return the response
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response if response else "🤖 Sorry, I didn't understand that."

# -----------------------------
# Example test cases for inference
# -----------------------------
print("🗣 AI Therapist:", generate_response("Hello, how are you?"))
print("🗣 AI Therapist:", generate_response("I'm feeling very sad."))
print("🗣 AI Therapist:", generate_response("What should I do if I'm stressed?"))

🗣 AI Therapist: Therapist: Hello, how are you? Advice: I want to hear from you about your experience with therapy and help you understand what's going through your mind.
🗣 AI Therapist: Therapist: I know you're feeling sad. Can you tell me more about what's making you feel so sad?
🗣 AI Therapist: Therapist: What should I do if I'm stressed?


Link Github to Google Colab code